In [1]:
from bs4 import BeautifulSoup as bs
import urllib
import re
import string
import pandas as pd
from operator import itemgetter

In [22]:
bi = pd.read_csv('../Data/List_of_British_Invasion_artists.csv')

In [21]:
bi_list = list(bi.artist.unique())

In [30]:
charts = pd.read_csv('../Data/1964_1969_Billboard_Charts.csv', encoding='utf-8')

In [19]:
chart_list = list(charts.artist.unique())

In [23]:
artists_to_fetch = []
for item in chart_list:
    if item in bi_list:
        pass
    else:
        artists_to_fetch.append(item)
        #print item

Bobby Vinton
The Kingsmen
The Singing Nun (Soeur Sourire)
Lenny Welch
Bobby Rydell
The Murmaids
Johnny Tillotson
Martha & The Vandellas
Shirley Ellis
Joey Powers
The Caravelles
Dion Di Muci
The Trashmen
Brenda Lee
Jack Jones
Roy Orbison
Jan & Dean
The Beach Boys
Nino Tempo & April Stevens
The Secrets
Dale & Grace
Chubby Checker
Trini Lopez
Marvin Gaye
Tommy Roe
Ray Charles
Dick and DeeDee
The Supremes
Garnet Mimms & The Enchanters
The Duprees featuring Joey Vann
The Tymes
Dionne Warwick
The Marketts
Jerry Butler
The Rip Chords
The Chiffons
Peter, Paul & Mary
The Miracles
Jimmy Gilmer And The Fireballs
Rick Nelson
The Ronettes
Robin Ward
Dean And Jean
The Chad Mitchell Trio
The Cookies
Lesley Gore
Rufus Thomas
The Marvelettes
Neil Sedaka
Murry Kellum
Steve & Eydie
Cliff Richard
Major Lance
The Dynamics
Mary Wells
Henry Mancini And His Orchestra
The Tams
The Orlons
The Raindrops
The Kingston Trio
Brooks O'Dell
The Baja Marimba Band
The Girlfriends
Connie Francis
Ginny Arnell
Burl Ives
To

In [24]:
len(chart_list)

1305

In [61]:
len(artists_to_fetch)
charts = charts[charts.artist.isin(artists_to_fetch)]

In [62]:
import pandasql as ps

In [63]:
q = '''
    select artist, count(song) as total_top_100 from
    (select distinct artist, song from charts)
    group by artist 
    order by count(song) asc
    '''

In [64]:
Artists_total_100 = ps.sqldf(q, globals())

In [79]:
artists_to_fetch = Artists_total_100["artist"][Artists_total_100["total_top_100"] > 10].values

In [80]:
len(artists_to_fetch)

87

In [67]:
def url(artist):
    base = "http://www.lyricsfreak.com/"
    name = artist.lower().strip().translate(None, string.punctuation).split(" ")
    if '' in name:
        name.remove('')
    init = name[0][0]
    search_name = "+".join(name)
    URL = base + init + "/" + search_name + "/"
    return URL

def fetch(artist):
    base = "http://www.lyricsfreak.com"
    song_list = []
    URL = url(artist)
    page = urllib.urlopen(URL)
    soup = bs(page, "lxml")
    songs = soup.findAll("table", attrs={"name": "song"})[0].findAll("tr")
    for s in songs:
        for n in s.findAll("td", attrs={"class": "colfirst"}):
            name = re.sub(" Lyrics","",n.find('a').attrs["title"])
            lyric_link = base + n.find('a').attrs["href"]
            song_list.append({name:lyric_link})
    return song_list

def fetch_all_artists(artist_list):
    lists = {}
    for i, artist in enumerate(artist_list):
        # print artist
        # print "\r{}%".format(100.0*(i+1)/len(artist_list)),
        song_list = []
        if artist[:3].lower() == 'the':
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(artist[4:])
            except IndexError:
                pass
        elif 'and' in artist.split(" "):
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(re.sub(' and ', ' & ', artist))
            except IndexError:
                pass
        else:
            try:
                song_list = fetch(artist)
            except:
                print "[Warning!] {}: Not Exist".format(artist)
        lists[artist] = song_list
    return lists

def ly_correct_url(URL):
    # input: lyric_page_url
    # output: lyric_correct_url
    page = urllib.urlopen(URL)
    soup = bs(page, 'lxml')
    base = 'http://www.lyricsfreak.com'
    try:
        correct_url = base + soup.find_all('a', attrs={'class': 'lb-correct'})[0].attrs['href']
    except IndexError:
        correct_url = None
    return correct_url

def fetch_lyrics(URL):
    # input: lyric_page_url
    URL2 = ly_correct_url(URL)
    if URL2:
        page = urllib.urlopen(URL2)
        soup = bs(page, 'lxml')
        lyric_text = soup.findAll('textarea', attrs={'name': 'lyrics'})[0].text
        return lyric_text
    else:
        return None

def fetch_one_artist_all_lyrics(artist):
    li =  [artist] #;print li
    d = fetch_all_artists(li) #;print d
    song_list = d[artist]
    lyrics_list = {} # {name: lyrics}
    for i, song in enumerate(song_list):
        # print '\r{}%'.format(100.0*(i+1)/len(song_list)),
        name = song.keys()[0] #;print name
        url = song.values()[0] #;print url
        lyrics = fetch_lyrics(url)
        lyrics_list[name] = lyrics
    return lyrics_list

def fetch_all_artists_all_lyrics(artist_list):
    artists_songs_lyrics = {} # {artist: {song: lyrics}}
    for i, artist in enumerate(artist_list):
        songs_lyrics = fetch_one_artist_all_lyrics(artist)
        artists_songs_lyrics[artist] = songs_lyrics
        print '\r{}%'.format(100.0*(i+1)/len(artist_list)),
    return artists_songs_lyrics

In [83]:
#res = fetch_all_artists_all_lyrics(artists_to_fetch)

[Warning!] Bee Gees: Not Exist
1.14942528736% [Warning!] Bob Dylan: Not Exist
2.29885057471% [Warning!] Brook Benton: Not Exist
3.44827586207% [Warning!] Carla Thomas: Not Exist
4.59770114943% [Warning!] Gerry And The Pacemakers: Not Exist
5.74712643678% [Warning!] Gladys Knight And The Pips: Not Exist
6.89655172414% [Warning!] Glen Campbell: Not Exist
8.04597701149% [Warning!] Smokey Robinson & The Miracles: Not Exist
9.19540229885%

TypeError: translate() takes exactly one argument (2 given)

In [81]:
artists_to_fetch

array([u'Bee Gees', u'Bob Dylan', u'Brook Benton', u'Carla Thomas',
       u'Gerry And The Pacemakers', u'Gladys Knight And The Pips',
       u'Glen Campbell', u'Smokey Robinson & The Miracles',
       u'The 5th Dimension', u'The Association', u'The Dells',
       u'The Grass Roots', u'The Lettermen', u'The Miracles',
       u'The Shangri-Las', u'Bobby Darin', u"Booker T. & The MG's",
       u'James Brown', u'Joe Simon', u'Johnny Cash', u'Mary Wells',
       u'Peter, Paul & Mary', u'Roy Orbison', u'Sam Cooke',
       u'The Drifters', u'Trini Lopez', u'Gene Pitney', u'Jackie Wilson',
       u'Jan & Dean', u'Martha & The Vandellas',
       u'Paul Revere & The Raiders Featuring Mark Lindsay',
       u'Percy Sledge', u'Sam & Dave', u'Tommy Roe', u'Andy Williams',
       u'Eddy Arnold', u'Gene Chandler', u'Peter And Gordon',
       u'Sonny & Cher', u"The Lovin' Spoonful", u'The Mamas & The Papas',
       u'The Vogues', u'Connie Francis', u'Gary Lewis And The Playboys',
       u'Jay & The Am

In [82]:
bi.values

array([[0, 'The Animals'],
       [1, 'The Beatles'],
       [2, 'Cilla Black'],
       [3, 'Chad & Jeremy'],
       [4, 'Cream'],
       [5, 'Dave Dee, Dozy, Beaky, Mick & Tich'],
       [6, 'The Dave Clark Five'],
       [7, 'Petula Clark'],
       [8, 'The Spencer Davis Group'],
       [9, 'Donovan'],
       [10, 'Adam Faith'],
       [11, 'Marianne Faithfull'],
       [12, 'Georgie Fame'],
       [13, 'Wayne Fontana and the Mindbenders'],
       [14, 'The Fortunes'],
       [15, 'The Fourmost'],
       [16, 'Freddie and the Dreamers'],
       [17, 'Gerry and the Pacemakers'],
       [18, "Herman's Hermits"],
       [19, 'The Hollies'],
       [20, 'The Honeycombs'],
       [21, 'The Hullaballoos'],
       [22, 'The Ivy League'],
       [23, 'Tom Jones'],
       [24, 'Jonathan King'],
       [25, 'The Kinks'],
       [26, 'Billy J. Kramer & The Dakotas'],
       [27, 'Lulu'],
       [28, 'Manfred Mann'],
       [29, 'The Moody Blues'],
       [30, 'The Nashville Teens'],
       [31,